In [1]:
import requests
import pandas as pd
import censusdis.data as ced
from censusdis import states
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from pycaret.regression import *

In [298]:
#variables of interest
download_variables=['NAME', 'B15003_022E', 'B25077_001E']


In [299]:
#download data
df_2022 = ced.download(
    
    # Data set: American Community Survey 5-Year
    dataset='acs/acs5',
    
    # Vintage: 2022
    vintage=2022, 
    
    # Variable: median household income
    download_variables=download_variables, 
    
    # Geography: Georgia State
    state=states.GA,
    county="*",
    
    with_geometry=True
)

df_2022.head()


,STATE,COUNTY,NAME,B15003_022E,B25077_001E,geometry
0,13,001,"Appling County, Georgia",676,88500,"POLYGON ((-82.55069 31.74911, -82.54744 31.749..."
1,13,003,"Atkinson County, Georgia",306,77100,"POLYGON ((-83.14100 31.40673, -83.13898 31.408..."
2,13,005,"Bacon County, Georgia",373,95600,"POLYGON ((-82.62819 31.56593, -82.62734 31.672..."
3,13,007,"Baker County, Georgia",227,121600,"POLYGON ((-84.64166 31.31250, -84.63994 31.312..."
4,13,009,"Baldwin County, Georgia",3770,154400,"POLYGON ((-83.42674 33.18273, -83.42496 33.182..."


In [300]:
#add year column with value 2022 for the entire dataset
df_2022['year'] = 2022

In [301]:
#change variable name to more descriptive names B15003_022E = median_bachelor, B25077_001E = median_income
df_2022.rename(columns={'B15003_022E': 'median_bachelor_edu', 'B25077_001E': 'median_home_value'}, inplace=True)

In [302]:
#Get data for 2018
df_2018 = ced.download(
    
    # Data set: American Community Survey 5-Year
    dataset='acs/acs5',
    
    # Vintage: 2018
    vintage=2018, 
    
    # Variable: median household income
    download_variables=download_variables, 
    
    # Geography: Georgia State
    state=states.GA,
    county="*",
    
    with_geometry=True
)

#add year column with value 2018 for the entire dataset
df_2018['year'] = 2018

#rename variables
df_2018.rename(columns={'B15003_022E': 'median_bachelor_edu', 'B25077_001E': 'median_home_value'}, inplace=True)

df_2018.head()


,STATE,COUNTY,NAME,median_bachelor_edu,median_home_value,geometry,year
0,13,005,"Bacon County, Georgia",439,73000,"POLYGON ((-82.62819 31.56593, -82.62734 31.672...",2018
1,13,025,"Brantley County, Georgia",557,68200,"POLYGON ((-82.28456 31.22445, -82.28371 31.226...",2018
2,13,017,"Ben Hill County, Georgia",843,80100,"POLYGON ((-83.48453 31.83552, -83.48312 31.838...",2018
3,13,033,"Burke County, Georgia",1111,87900,"POLYGON ((-82.31651 32.83577, -82.29147 32.888...",2018
4,13,047,"Catoosa County, Georgia",6259,140000,"POLYGON ((-85.26505 34.98508, -85.25495 34.985...",2018


In [304]:
#arrange both datasets with ascending order of TRACT variable
df_2018 = df_2018.sort_values(by='COUNTY')
df_2022 = df_2022.sort_values(by='COUNTY')


In [305]:
#create change_median_bachelor_edu and change_median_home_value
df_2022['change_median_bachelor_edu'] = (df_2022['median_bachelor_edu'] - df_2018['median_bachelor_edu']) / df_2018['median_bachelor_edu']
df_2022['change_median_home_value'] = (df_2022['median_home_value'] - df_2018['median_home_value']) / df_2018['median_home_value']


In [306]:
#shift all values in change_median_bachelor_edu and change_median_home_value columns to ensure positive values

#---change_median_bachelor_edu---
# Step 1: Find the minimum value in the change_median_bachelor_edu column
min_value_bachelor_edu = df_2022['change_median_bachelor_edu'].min()

# Step 2: Adjust the minimum value by adding 1
adjusted_min_value = abs(min_value_bachelor_edu) + 1

# Step 3: Add this adjusted value to all values in the column
df_2022['change_median_bachelor_edu'] += adjusted_min_value

#---change_median_home_value---
# Step 1: Find the minimum value in the change_median_home_value column
min_value_home_value = df_2022['change_median_home_value'].min()

# Step 2: Adjust the minimum value by adding 1
adjusted_min_value_home = abs(min_value_home_value) + 1

# Step 3: Add this adjusted value to all values in the column
df_2022['change_median_home_value'] += adjusted_min_value_home



In [307]:
#scale the change_median_bachelor_edu and change_median_home_value columns by dividing by the maximum value in the column
df_2022['change_median_bachelor_edu'] = df_2022['change_median_bachelor_edu'] / df_2022['change_median_bachelor_edu'].max()
df_2022['change_median_home_value'] = df_2022['change_median_home_value'] / df_2022['change_median_home_value'].max()

In [308]:
#create new column called gentrification_index where we will multiply the two change columns
df_2022['gentrification_index'] = df_2022['change_median_bachelor_edu'] * df_2022['change_median_home_value']



In [309]:
df_2022.head()

,STATE,COUNTY,NAME,median_bachelor_edu,median_home_value,geometry,year,change_median_bachelor_edu,change_median_home_value,gentrification_index
0,13,001,"Appling County, Georgia",676,88500,"POLYGON ((-82.55069 31.74911, -82.54744 31.749...",2022,0.005771,0.295071,0.001703
1,13,003,"Atkinson County, Georgia",306,77100,"POLYGON ((-83.14100 31.40673, -83.13898 31.408...",2022,0.003517,0.281585,0.000990
2,13,005,"Bacon County, Georgia",373,95600,"POLYGON ((-82.62819 31.56593, -82.62734 31.672...",2022,0.003274,0.291970,0.000956
3,13,007,"Baker County, Georgia",227,121600,"POLYGON ((-84.64166 31.31250, -84.63994 31.312...",2022,0.002732,0.323265,0.000883
4,13,009,"Baldwin County, Georgia",3770,154400,"POLYGON ((-83.42674 33.18273, -83.42496 33.182...",2022,0.003637,0.277029,0.001008


In [310]:
df_2022.describe()

,median_bachelor_edu,median_home_value,year,change_median_bachelor_edu,change_median_home_value,gentrification_index
count,159.000000,159.000000,159.0,159.000000,159.000000,159.000000
mean,9211.251572,165625.786164,2022.0,0.027109,0.363840,0.016540
std,28723.604828,76881.856771,0.0,0.089827,0.175393,0.064204
min,86.000000,65300.000000,2022.0,0.002275,0.164815,0.000398
25%,648.500000,106050.000000,2022.0,0.002963,0.240264,0.000726
50%,1671.000000,149400.000000,2022.0,0.004859,0.308125,0.001393
75%,5264.000000,206450.000000,2022.0,0.012439,0.438716,0.005213
max,242267.000000,451100.000000,2022.0,1.000000,1.000000,0.729943


In [311]:
# One hot encoding for the NAME column and put in 0 if false and 1 if true
df_2022 = pd.get_dummies(df_2022, columns=['NAME'])

#rename new columns by removing the NAME_ prefix
df_2022.columns = df_2022.columns.str.replace('NAME_', '')

In [313]:
#drop columns that are not needed
df_2022 = df_2022.drop(columns=['geometry', 'STATE', 'COUNTY', 'year', 'median_bachelor_edu', 'median_home_value', 'change_median_bachelor_edu', 'change_median_home_value'])

In [314]:
#split the data in training and test

X = df_2022.drop(columns=['gentrification_index'])
y = df_2022['gentrification_index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [315]:
#initialize the models
rf = RandomForestRegressor()
lr = LinearRegression()

#fit the models
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

#predict
rf_pred = rf.predict(X_test)
lr_pred = lr.predict(X_test)

In [316]:
#Evaluate the models

#calculate the percentage error
rf_error = abs(rf_pred - y_test) / y_test
lr_error = abs(lr_pred - y_test) / y_test

#calculate the mean percentage error
rf_mean_error = rf_error.mean()
lr_mean_error = lr_error.mean()

print(f'Random Forest Mean Percentage Error: {rf_mean_error}')

print(f'Linear Regression Mean Percentage Error: {lr_mean_error}')

Random Forest Mean Percentage Error: 0.566122175328132
Linear Regression Mean Percentage Error: 32.51564780596694
